In [4]:
import pandas as pd
import gzip 
import IPython.display
#from tabulate import tabulate

In [5]:
from IPython.display import display, HTML
display(HTML('<style>.container {width: 100%} !i'))

In [6]:
# Define the column headers for the ALB access logs DataFrame
columns = [
    "proto",                                              # skip
    "timestamp",                    #          2023-03-04T00:40:00.673223Z               
    "elb",                                                # skip
    "client:port",                                        # 18.197.198.192:5310
    "target:port",                                        # skip
    "request_processing_time",                            # skip
    "latency",                      # float32 (?)         0.110
    "response_processing_time",                           # skip
    "status",                       # string(int16)         200
    "target_status",                                      # 200
    "r_bytes",                      # int16                  702
    "s_bytes",                      # int16                  537
    "request",                      # string (?)         POST https://pc-energy.encore-prod.enbw.cloud:443/service/Electricity HTTP/1.1
    "user_agent",                                         # skip
    "ssl_cipher",                                         # skip
    "ssl_protocol",                                       # skip
    "target_group_arn",                                   # skip
    "trace_id",                                           # skip
    "domain_name",                                        # skip
    "chosen_cert_arn",                                    # skip
    "matched_rule_priority",                              # skip
    "request_creation_time",        
    "actions_executed",                                   # skip
    "r_url",                        # string                 - 
    "error_reason",                                       # skip
    "target_ip",                                          # skip
    "target_status_description",                          # skip
    "code",                         # float64 (?)         NaN   jedenfalls am Anfang  
    "target_response_duration",                           # skip
    "target_health_description"                           # skip
]

In [7]:
FILE = '/BigData/ELB/2018/12/11/647933830095_elasticloadbalancing_eu-central-1_app.awseb-AWSEB-18IHG6R3W1A7K.ac573d33cdf8d1a2_20181211T2355Z_35.157.194.235_tbc04iw9.log.gz'
with gzip.open(FILE, 'rt') as file:
    df = pd.read_csv(FILE,  sep=' ', names=columns, quotechar='"', engine='python')

In [8]:
df.drop(["proto", "elb", "target_status", "client:port", "target:port", "request_processing_time", "response_processing_time", "user_agent", "ssl_cipher", "ssl_protocol", "target_group_arn"], axis=1, inplace=True)

In [9]:
df.drop(["trace_id",  "domain_name", "chosen_cert_arn",  "matched_rule_priority", "request_creation_time", "actions_executed"], axis=1, inplace=True)

In [10]:
df.drop(["error_reason", "target_ip", "target_status_description","target_response_duration", "target_health_description"], axis=1, inplace=True)

In [11]:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 100)

In [12]:
pd.set_option('display.max_rows', 10)

In [13]:
df.head(10)

,timestamp,latency,status,r_bytes,s_bytes,request,r_url,code
0,2018-12-11T23:50:10.814742Z,0.014,200,363,752,POST https://powercloud.enpower.de:443/service:api/get-product HTTP/1.1,-,NaN
1,2018-12-11T23:50:27.900449Z,0.108,200,396,566,GET https://powercloud.enpower.de:443/rest:client/a88afc059eba6bf3dfaa73b714c6e113/getBICFromIBA...,-,NaN
2,2018-12-11T23:50:27.900581Z,0.108,200,401,594,GET https://powercloud.enpower.de:443/rest:client/a88afc059eba6bf3dfaa73b714c6e113/getBankNameFr...,-,NaN
3,2018-12-11T23:50:29.538233Z,0.547,200,4522,504,POST https://powercloud.enpower.de:443/rest:client/a88afc059eba6bf3dfaa73b714c6e113/createOrder ...,-,NaN
4,2018-12-11T23:50:32.032335Z,0.100,200,505,512,GET https://powercloud.enpower.de:443/rest:customer/a88afc059eba6bf3dfaa73b714c6e113/getInvoices...,-,NaN
5,2018-12-11T23:50:32.649405Z,0.116,200,431,983,GET https://powercloud.enpower.de:443/rest:client/a88afc059eba6bf3dfaa73b714c6e113/getPricesForA...,-,NaN
6,2018-12-11T23:50:34.347581Z,0.122,200,504,1007,GET https://powercloud.enpower.de:443/rest:customer/a88afc059eba6bf3dfaa73b714c6e113/getInvoices...,-,NaN
7,2018-12-11T23:50:34.673519Z,0.097,200,504,949,GET https://powercloud.enpower.de:443/rest:customer/a88afc059eba6bf3dfaa73b714c6e113/getDeposits...,-,NaN
8,2018-12-11T23:50:35.056182Z,0.115,200,450,502,GET https://powercloud.enpower.de:443/rest:client/a88afc059eba6bf3dfaa73b714c6e113/getNextUseAtD...,-,NaN
9,2018-12-11T23:50:35.227155Z,0.019,200,196,7726,GET https://powercloud.enpower.de:443/login HTTP/1.1,-,NaN


In [14]:
pd.get_option("display.max_columns")
pd.get_option("display.max_rows")

10

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2375 entries, 0 to 2374
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  2375 non-null   object 
 1   latency    2375 non-null   float64
 2   status     2375 non-null   int64  
 3   r_bytes    2375 non-null   int64  
 4   s_bytes    2375 non-null   int64  
 5   request    2375 non-null   object 
 6   r_url      2375 non-null   object 
 7   code       0 non-null      float64
dtypes: float64(2), int64(3), object(3)
memory usage: 148.6+ KB


In [16]:
df['latency'] = df['latency'].clip(upper=255)
df['latency'] = df['latency'].astype('float16')
df['status'] = df['status'].astype('uint16')
df['r_bytes'] = df['r_bytes'].astype('uint32')
df['s_bytes'] = df['s_bytes'].astype('uint32')

In [17]:
split_fields = df['request'].str.split(' ', expand=True)
df['method'] = split_fields[0]
df['url'] = split_fields[1]
df['protocol'] = split_fields[2]

In [18]:
df.drop('request', axis=1, inplace=True)

In [19]:
df['url'] = df['url'].astype(str)

In [20]:
df['subpath'] = df['url'].str.split('443/').str[1].str.split('/').str[:2].str.join('/')
df = df.reindex(columns=['timestamp', 'latency', 'status', 'r_bytes', 's_bytes', 'subpath', 'method', 'url', 'protocol', 'r_url', 'code'])

In [21]:
header_style = [
    {'selector': 'th', 'props': [('text-align', 'center')]}
]
styled_df = df.head(10).style.set_properties(subset=['url', 'subpath'], **{'text-align': 'left'})
styled_df = styled_df.set_table_styles(header_style)
styled_df


,timestamp,latency,status,r_bytes,s_bytes,subpath,method,url,protocol,r_url,code
0,2018-12-11T23:50:10.814742Z,0.014000,200,363,752,service:api/get-product,POST,https://powercloud.enpower.de:443/service:api/get-product,HTTP/1.1,-,nan
1,2018-12-11T23:50:27.900449Z,0.107971,200,396,566,rest:client/a88afc059eba6bf3dfaa73b714c6e113,GET,https://powercloud.enpower.de:443/rest:client/a88afc059eba6bf3dfaa73b714c6e113/getBICFromIBAN?iban=DE18100000000016001503,HTTP/1.1,-,nan
2,2018-12-11T23:50:27.900581Z,0.107971,200,401,594,rest:client/a88afc059eba6bf3dfaa73b714c6e113,GET,https://powercloud.enpower.de:443/rest:client/a88afc059eba6bf3dfaa73b714c6e113/getBankNameFromIBAN?iban=DE18100000000016001503,HTTP/1.1,-,nan
3,2018-12-11T23:50:29.538233Z,0.546875,200,4522,504,rest:client/a88afc059eba6bf3dfaa73b714c6e113,POST,https://powercloud.enpower.de:443/rest:client/a88afc059eba6bf3dfaa73b714c6e113/createOrder,HTTP/1.1,-,nan
4,2018-12-11T23:50:32.032335Z,0.099976,200,505,512,rest:customer/a88afc059eba6bf3dfaa73b714c6e113,GET,https://powercloud.enpower.de:443/rest:customer/a88afc059eba6bf3dfaa73b714c6e113/getInvoicesByContractId?contractId=70100272566&limit=9999&offset=0,HTTP/1.1,-,nan
5,2018-12-11T23:50:32.649405Z,0.116028,200,431,983,rest:client/a88afc059eba6bf3dfaa73b714c6e113,GET,https://powercloud.enpower.de:443/rest:client/a88afc059eba6bf3dfaa73b714c6e113/getPricesForAddOnProductByCode?productCode=PHILIPS_TV_43_NEU&limit=9999&offset=0,HTTP/1.1,-,nan
6,2018-12-11T23:50:34.347581Z,0.122009,200,504,1007,rest:customer/a88afc059eba6bf3dfaa73b714c6e113,GET,https://powercloud.enpower.de:443/rest:customer/a88afc059eba6bf3dfaa73b714c6e113/getInvoicesByContractId?contractId=1811343486&limit=9999&offset=0,HTTP/1.1,-,nan
7,2018-12-11T23:50:34.673519Z,0.096985,200,504,949,rest:customer/a88afc059eba6bf3dfaa73b714c6e113,GET,https://powercloud.enpower.de:443/rest:customer/a88afc059eba6bf3dfaa73b714c6e113/getDepositsByContractId?contractId=1811343486&limit=9999&offset=0,HTTP/1.1,-,nan
8,2018-12-11T23:50:35.056182Z,0.114990,200,450,502,rest:client/a88afc059eba6bf3dfaa73b714c6e113,GET,https://powercloud.enpower.de:443/rest:client/a88afc059eba6bf3dfaa73b714c6e113/getNextUseAtDepositByContractId?contractId=1811343486,HTTP/1.1,-,nan
9,2018-12-11T23:50:35.227155Z,0.018997,200,196,7726,login,GET,https://powercloud.enpower.de:443/login,HTTP/1.1,-,nan


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2375 entries, 0 to 2374
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  2375 non-null   object 
 1   latency    2375 non-null   float16
 2   status     2375 non-null   uint16 
 3   r_bytes    2375 non-null   uint32 
 4   s_bytes    2375 non-null   uint32 
 5   subpath    2375 non-null   object 
 6   method     2375 non-null   object 
 7   url        2375 non-null   object 
 8   protocol   2375 non-null   object 
 9   r_url      2375 non-null   object 
 10  code       0 non-null      float64
dtypes: float16(1), float64(1), object(6), uint16(1), uint32(2)
memory usage: 157.8+ KB


In [23]:
max_r_bytes = df['r_bytes'].max()
max_s_bytes = df['s_bytes'].max()

print("Max r_bytes:", max_r_bytes)
print("Max s_bytes:", max_s_bytes)

Max r_bytes: 56280
Max s_bytes: 94821


In [24]:
latency_stats = df['latency'].describe()
print(latency_stats)

count    2375.000000
mean        0.142334
std         0.142578
min         0.006001
25%         0.109009
50%         0.116028
75%         0.128052
max         2.888672
Name: latency, dtype: float64


In [25]:
latency_median = df['latency'].median()
latency_95th_percentile = df['latency'].quantile(0.95)
print(latency_95th_percentile)

0.23430175781249973


In [26]:
subpath_avg_latency = df.groupby('subpath')['latency'].mean()
sorted_subpaths = subpath_avg_latency.sort_values(ascending=False)
top_5_subpaths = sorted_subpaths.head(5)
print(top_5_subpaths)

subpath
rest:client/72e48c67dcb095f52fccfaa22189ca3c      0.234543
rest:customer/a88afc059eba6bf3dfaa73b714c6e113    0.145498
rest:client/06b86119ad12044bb7fd680cd568a8a6      0.143411
rest:client/a88afc059eba6bf3dfaa73b714c6e113      0.137258
rest:client/2cee36e616b3eac6976c4a30390ba2a3      0.136464
Name: latency, dtype: float32


In [28]:
highest_latencies = df.nlargest(5, 'latency')
print(highest_latencies)


                        timestamp   latency  status  r_bytes  s_bytes                                         subpath method                                                                                                  url  protocol r_url  code
2073  2018-12-11T23:53:03.517736Z  2.888672     200      558      519    rest:client/06b86119ad12044bb7fd680cd568a8a6   POST  https://powercloud.enpower.de:443/rest:client/06b86119ad12044bb7fd680cd568a8a6/linkEdiMailToArch...  HTTP/1.1     -   NaN
2181  2018-12-11T23:53:03.504659Z  2.818359     200      552      513    rest:client/06b86119ad12044bb7fd680cd568a8a6   POST  https://powercloud.enpower.de:443/rest:client/06b86119ad12044bb7fd680cd568a8a6/linkEdiMailToArch...  HTTP/1.1     -   NaN
2071  2018-12-11T23:53:03.455480Z  2.812500     200      558      519    rest:client/06b86119ad12044bb7fd680cd568a8a6   POST  https://powercloud.enpower.de:443/rest:client/06b86119ad12044bb7fd680cd568a8a6/linkEdiMailToArch...  HTTP/1.1     -   NaN
1985  20